In [ ]:
from webutils import get_title_url_year
# 文章的google scholar 链接
url_scholar = 'https://scholar.google.com/scholar?cites=2647023382806321658&as_sdt=2005&sciodt=0,5&hl=zh-TW'
get_title_url_year(url_scholar)

In [ ]:
import pandas as pd
from utils import Paper

# 读取检索到的所有文章的title，url，year
df = pd.read_csv('data/all_articles.csv')
papers = []
for index, row in df.iterrows():
    title = row[0]
    url = row[1]
    year = row[2]
    papers.append(Paper(title, url, year=year))


In [ ]:
from webutils import arxiv,nips,acl,openreview,ieee
import csv

arxiv([p for p in papers if 'arxiv' in p.url])
nips([p for p in papers if 'neurips' in p.url])
acl([p for p in papers if 'aclanthology' in p.url])
openreview([p for p in papers if 'openreview' in p.url])
ieee([p for p in papers if 'ieee.org' in p.url])

seen = set()
with open("data/papers_with_author.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "year"])
    keywords = {'arxiv', 'neurips', 'aclanthology', 'openreview', 'ieee.org'}
    for paper in (p for p in papers if any(keyword in p.url for keyword in keywords)):
        author_list = []
        for author in paper.authors:
            if author not in seen:
                seen.add(author)
                author_list.append(author)
        writer.writerow([paper.title, paper.url, author_list, paper.year])
        # 保存'arxiv', 'neurips', 'aclanthology', 'openreview', 'ieee.org' 的相关信息,其他出处文章请自行查找并加入

In [ ]:
import ast
from homepage_utils import get_author_homepage

df = pd.read_csv('data/papers_with_author.csv')
papers = []
print(len(df))
for index, row in df.iterrows():
    title = row[0]
    url = row[1]
    str = row[2]
    year = row[3]
    author_list = ast.literal_eval(str)
    print(author_list)
    papers.append(Paper(title, url, authors=author_list, year=year))
for paper in papers:
    for author in paper.authors:
        homepage = get_author_homepage(author,api_key="010e0726689c76ab1bd7aee362f28a55a24ae18f")
        print(homepage)
        paper.author_homepages.append(homepage)
with open("data/papers_with_homepage.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "Authors-HomePage", "Year-Source"])
    for paper in [p for p in papers]:
        writer.writerow([paper.title, paper.url, paper.authors, paper.author_homepages, paper.year])

In [ ]:
from find_citation import extract_text_with_fitz,find_citation_symbol,find_content
from utils import read_data
import os
import csv
keywords = {
            'Yu et al., 2023','Yu et al.,2023','Yu et al.,\n2023','Yu et al.,\n 2023','Yu et al., \n2023', 'Yu et al., 2024','Yu et al.,2024','Yu et al.,\n2024','Yu et al.,\n 2024','Yu et al., \n2024',
            'Yu et al., (2023)','Yu et al.,(2023)','Yu et al.,\n(2023)','Yu et al.,\n (2023)','Yu et al., \n(2023)', 'Yu et al., (2024)','Yu et al.,(2024)','Yu et al.,\n(2024)','Yu et al.,\n (2024)','Yu et al., \n(2024)',
            'Yu et al. 2023','Yu et al.2023','Yu et al.\n2023','Yu et al.\n 2023','Yu et al. \n2023', 'Yu et al. 2024','Yu et al.2024','Yu et al.\n2024','Yu et al.\n 2024','Yu et al. \n2024',
            'Yu et al. (2023)','Yu et al.(2023)','Yu et al.\n(2023)','Yu et al.\n (2023)','Yu et al. \n(2023)', 'Yu et al. (2024)','Yu et al.(2024)','Yu et al.\n(2024)','Yu et al.\n (2024)','Yu et al. \n(2024)',             
            }

papers = read_data('./data/combined_result.csv')

for paper in papers:
    if 'arxiv' not in paper.url:
        continue
    arxiv_id = paper.url.split('/')[-1]
    arxiv_pdf = os.path.join('downloads',f'{arxiv_id}.pdf')
    text = extract_text_with_fitz(arxiv_pdf).replace('\n','')
    content = find_content(text,keywords)
    symbol = find_citation_symbol(content,keywords)
    paper.set_content(content)
    paper.set_symbol(symbol)

with open("data/papers_with_content.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Authors", "Authors-HomePage", "Year-Source", "Content", "Symbol"])
    for paper in [p for p in papers]:
        writer.writerow([paper.title, paper.url, paper.authors, paper.author_homepages, paper.year, paper.content, paper.symbol])


In [ ]:
from pptutils import generate
from utils import read_data

papers = read_data('./data/papers_with_content.csv')
generate(papers,'./data/test.pptx')